<a href="https://colab.research.google.com/github/OluwafemiOlasupo/Coding-Contest/blob/main/Problem_2_Laptop_Product_Lineup_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
# Replace 'laptop_data.csv' with the actual path to your downloaded dataset
df = pd.read_csv('laptop_data.csv')

In [ ]:
# Display basic info about the dataset
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())


Dataset shape: (1000, 6)

Column names:
['Memory', 'Storage', 'CPU_class', 'Screen_size', 'year', 'price']

First few rows:
   Memory  Storage  CPU_class  Screen_size  year   price
0       8     1024          3         16.0  2025  162021
1      32     1024          0         16.0  2024  112214
2      16      512          1         16.0  2025  118354
3      16     1024          1         16.0  2025  123383
4      16      256          3         16.0  2024  162968


In [ ]:
# Current model specifications
current_specs = {
    'memory_gb': 16,
    'storage_gb': 512,
    'cpu_class': 1,
    'screen_size': 14.0,
    'release_year': 2025
}

current_price = 111000  # yen

In [ ]:
# Upgrade options and their costs
upgrade_options = {
    'memory_upgrade': {'cost': 7000, 'spec_change': {'memory_gb': 32}},  # +16GB
    'storage_upgrade': {'cost': 5000, 'spec_change': {'storage_gb': 1024}},  # +512GB
    'cpu_upgrade': {'cost': 15000, 'spec_change': {'cpu_class': 2}},  # +1 level
    'screen_upgrade': {'cost': 3000, 'spec_change': {'screen_size': 16.0}}  # 14->16 inches
}

In [ ]:
def prepare_data(df):
    """Prepare and clean the dataset for analysis"""
    # Remove any rows with missing values in key columns
    required_columns = ['Memory', 'Storage', 'CPU_class', 'Screen_size', 'year', 'price']

    # Check which columns actually exist in the dataset
    existing_columns = [col for col in required_columns if col in df.columns]
    print(f"Available columns for analysis: {existing_columns}")

    # Clean the data
    df_clean = df[existing_columns].dropna()

    # Remove outliers (prices that are too extreme)
    price_col = 'price' if 'price' in df.columns else df.columns[-1]  # assume last column is price
    Q1 = df_clean[price_col].quantile(0.25)
    Q3 = df_clean[price_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_clean = df_clean[(df_clean[price_col] >= lower_bound) & (df_clean[price_col] <= upper_bound)]

    return df_clean

In [ ]:
def build_price_model(df_clean):
    """Build a regression model to predict laptop prices"""
    # Prepare features (X) and target (y)
    feature_columns = [col for col in df_clean.columns if col != 'price']
    if 'price' in df_clean.columns:
        price_column = 'price'
    else:
        price_column = df_clean.columns[-1]  # assume last column is price

    X = df_clean[feature_columns]
    y = df_clean[price_column]

    # Build and train the model
    model = LinearRegression()
    scaler = StandardScaler()

    X_scaled = scaler.fit_transform(X)
    model.fit(X_scaled, y)

    # Evaluate model
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y, y_pred)
    print(f"Model R² score: {r2:.3f}")

    return model, scaler, feature_columns

In [ ]:
def predict_price(model, scaler, feature_columns, specs):
    """Predict price for given specifications"""
    # Create feature vector matching the model's expected input
    feature_vector = []
    for col in feature_columns:
        if col in specs:
            feature_vector.append(specs[col])
        else:
            feature_vector.append(0)  # default value

    feature_vector = np.array(feature_vector).reshape(1, -1)
    feature_vector_scaled = scaler.transform(feature_vector)

    predicted_price = model.predict(feature_vector_scaled)[0]
    return predicted_price


In [ ]:
def analyze_upgrade_options(df, current_specs, current_price, upgrade_options):
    """Analyze each upgrade option and calculate expected gross profit"""
    # Prepare data and build model
    df_clean = prepare_data(df)
    model, scaler, feature_columns = build_price_model(df_clean)

    results = {}

    print("\n" + "="*60)
    print("UPGRADE ANALYSIS RESULTS")
    print("="*60)

    for upgrade_name, upgrade_info in upgrade_options.items():
        # Create upgraded specs
        upgraded_specs = current_specs.copy()
        upgraded_specs.update(upgrade_info['spec_change'])

        # Predict market price for upgraded model
        predicted_market_price = predict_price(model, scaler, feature_columns, upgraded_specs)

        # Calculate our selling price (current price + upgrade cost + margin)
        upgrade_cost = upgrade_info['cost']
        our_selling_price = current_price + upgrade_cost

        # Calculate gross profit (assuming we can sell at market price)
        # Gross profit = Market price - Our cost (current cost + upgrade cost)
        current_cost = current_price * 0.7  # Assuming 30% margin on current model
        upgraded_cost = current_cost + upgrade_cost
        gross_profit = predicted_market_price - upgraded_cost

        # Alternative calculation: profit per unit sold
        profit_margin = predicted_market_price - our_selling_price

        results[upgrade_name] = {
            'upgrade_cost': upgrade_cost,
            'predicted_market_price': predicted_market_price,
            'our_selling_price': our_selling_price,
            'gross_profit': gross_profit,
            'profit_margin': profit_margin,
            'price_increase_ratio': predicted_market_price / current_price
        }

        print(f"\n{upgrade_name.upper()}:")
        print(f"  Upgrade cost: ¥{upgrade_cost:,}")
        print(f"  Predicted market price: ¥{predicted_market_price:,.0f}")
        print(f"  Our selling price: ¥{our_selling_price:,}")
        print(f"  Expected gross profit: ¥{gross_profit:,.0f}")
        print(f"  Profit margin: ¥{profit_margin:,.0f}")
        print(f"  Market price increase: {(predicted_market_price/current_price-1)*100:.1f}%")

    return results


In [ ]:
def select_best_upgrades(results):
    """Select the top 2 upgrades based on gross profit"""
    # Sort by gross profit
    sorted_upgrades = sorted(results.items(), key=lambda x: x[1]['gross_profit'], reverse=True)

    print("\n" + "="*60)
    print("RANKING BY GROSS PROFIT")
    print("="*60)

    for i, (upgrade_name, metrics) in enumerate(sorted_upgrades, 1):
        print(f"{i}. {upgrade_name}: ¥{metrics['gross_profit']:,.0f}")

    top_2_upgrades = [upgrade[0] for upgrade in sorted_upgrades[:2]]

    print(f"\n🏆 TOP 2 RECOMMENDED UPGRADES:")
    print(f"1. {top_2_upgrades[0]}")
    print(f"2. {top_2_upgrades[1]}")

    return top_2_upgrades

In [ ]:
# Main analysis
def main():
    print("Starting laptop upgrade analysis...")

    # Load and analyze data
    try:
        df = pd.read_csv('laptop_data.csv')  # Replace with actual file path

        # Analyze upgrade options
        results = analyze_upgrade_options(df, current_specs, current_price, upgrade_options)

        # Select best upgrades
        best_upgrades = select_best_upgrades(results)

        return best_upgrades

    except FileNotFoundError:
        print("Please download the dataset and update the file path in the code.")
        print("Dataset URL: https://drive.google.com/file/d/1-PT29BP2ru2L9zQNNXK3y-kBZHxfrngn/view?usp=sharing")
        return None

if __name__ == "__main__":
    main()

Starting laptop upgrade analysis...
Available columns for analysis: ['Memory', 'Storage', 'CPU_class', 'Screen_size', 'year', 'price']
Model R² score: 0.995

UPGRADE ANALYSIS RESULTS

MEMORY_UPGRADE:
  Upgrade cost: ¥7,000
  Predicted market price: ¥-2,132,246
  Our selling price: ¥118,000
  Expected gross profit: ¥-2,216,946
  Profit margin: ¥-2,250,246
  Market price increase: -2020.9%

STORAGE_UPGRADE:
  Upgrade cost: ¥5,000
  Predicted market price: ¥-2,132,246
  Our selling price: ¥116,000
  Expected gross profit: ¥-2,214,946
  Profit margin: ¥-2,248,246
  Market price increase: -2020.9%

CPU_UPGRADE:
  Upgrade cost: ¥15,000
  Predicted market price: ¥-2,132,246
  Our selling price: ¥126,000
  Expected gross profit: ¥-2,224,946
  Profit margin: ¥-2,258,246
  Market price increase: -2020.9%

SCREEN_UPGRADE:
  Upgrade cost: ¥3,000
  Predicted market price: ¥-2,132,246
  Our selling price: ¥114,000
  Expected gross profit: ¥-2,212,946
  Profit margin: ¥-2,246,246
  Market price incre

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
